In [8]:
from pprint import pprint
import pandas as pd

filename = 'nwm.csv'
df = pd.read_csv(filename)
df = df.drop(df.columns[0], axis=1)

In [9]:
df

,C1,C2,C3,C4
0,0.60,0.93,0.00,0.73
1,0.66,0.55,0.45,0.49
2,1.00,0.45,0.57,0.50
3,0.48,0.87,0.00,0.75
4,0.62,0.40,0.56,0.50
5,0.78,0.27,0.71,0.50
6,0.40,0.90,0.00,0.82
7,0.64,0.44,0.54,0.54
8,0.65,0.30,0.71,0.55
9,0.45,0.86,0.00,0.73


Pary wariantów do oceny:
- 3 i 17 - para 15
- 11 i 14 - para 16
- 0 i 11 - para 9
- 13 i 25 - para 28 
- 7 i 8 - para 36

Nasza preferencja zakłda wybór wariantów gdzie oba kryteria, c3 i c4, są niższe. W efekcie:

- 3 jest nierozróżnialne z 17
- 11 jest preferowane nad 14
- 0 jest nierozróżnialne z 11
- 13 jest preferowane nad 25
- 7 jest preferowane nad 8



In [10]:
# Import the library
from pulp import *

# Create a problem variable:
prob = LpProblem("MaximizeEpsilon", LpMaximize)

# Create problem variables:
# epsilon
e = LpVariable("e", lowBound=0)

# mins
u1_32 = LpVariable("u1_32", lowBound=0) # u1_32>=0
u2_3 = LpVariable("u2_03", lowBound=0) # u2_03>=0
u3_0 = LpVariable("u3_0", lowBound=0) # u3_0>=0
u4_49 = LpVariable("u4_49", lowBound=0) # u4_49>=0

# maxs
u1_100 = LpVariable("u1_100", lowBound=0) # u1_100>=0
u2_100 = LpVariable("u2_100", lowBound=0) # u2_100>=0
u3_100 = LpVariable("u3_100", lowBound=0) # u3_100>=0
u4_100 = LpVariable("u4_100", lowBound=0) # u4_100>=0

# variant 3
u1_48 = LpVariable("u1_48", lowBound=0) # u1_48>=0
u2_87 = LpVariable("u2_87", lowBound=0) # u2_87>=0
# u3_0 = LpVariable("u3_0", lowBound=0) # u3_0>=0
u4_75 = LpVariable("u4_75", lowBound=0) # u4_75>=0

# variant 17
u1_76 = LpVariable("u1_76", lowBound=0) # u1_76>=0
u2_6 = LpVariable("u2_6", lowBound=0) # u2_6>=0
# u3_100 = LpVariable("u3_100", lowBound=0) # u3_100>=0
u4_60 = LpVariable("u4_60", lowBound=0) # u4_60>=0

# variant 11
u1_74 = LpVariable("u1_74", lowBound=0) # u1_74>=0
u2_25 = LpVariable("u2_25", lowBound=0) # u2_25>=0
u3_80 = LpVariable("u3_80", lowBound=0) # u3_80>=0
# u4_49 = LpVariable("u4_49", lowBound=0) # u4_49>=0

# variant 14
u1_87 = LpVariable("u1_87", lowBound=0) # u1_87>=0
# u2_03 = LpVariable("u2_03", lowBound=0) # u2_03>=0
# u3_100 = LpVariable("u3_100", lowBound=0) # u3_100>=0
u4_61 = LpVariable("u4_61", lowBound=0) # u4_61>=0

# variant 0
u1_60 = LpVariable("u1_60", lowBound=0) # u1_60>=0
u2_93 = LpVariable("u2_93", lowBound=0) # u2_93>=0
# u3_0 = LpVariable("u3_0", lowBound=0) # u3_0>=0
u4_73 = LpVariable("u4_73", lowBound=0) # u4_73>=0

# variant 13
u1_69 = LpVariable("u1_69", lowBound=0) # u1_69>=0
u2_49 = LpVariable("u2_49", lowBound=0) # u2_49>=0
u3_56 = LpVariable("u3_56", lowBound=0) # u3_56>=0
# u4_61 = LpVariable("u4_61", lowBound=0) # u4_61>=0

# variant 25
u1_71 = LpVariable("u1_71", lowBound=0) # u1_71>=0
# u2_25 = LpVariable("u2_25", lowBound=0) # u2_25>=0
u3_88 = LpVariable("u3_88", lowBound=0) # u3_88>=0
u4_67 = LpVariable("u4_67", lowBound=0) # u4_67>=0

# variant 7
u1_64 = LpVariable("u1_64", lowBound=0) # u1_64>=0
u2_44 = LpVariable("u2_44", lowBound=0) # u2_44>=0
u3_54 = LpVariable("u3_54", lowBound=0) # u3_54>=0
u4_54 = LpVariable("u4_54", lowBound=0) # u4_54>=0

# variant 8
u1_65 = LpVariable("u1_65", lowBound=0) # u1_65>=0
u2_30 = LpVariable("u2_30", lowBound=0) # u2_30>=0
u3_71 = LpVariable("u3_71", lowBound=0) # u3_71>=0
u4_55 = LpVariable("u4_55", lowBound=0) # u4_55>=0

u1 = [u1_32, u1_48, u1_60, u1_64, u1_65, u1_69, u1_71, u1_74, u1_76, u1_87, u1_100]
u2 = [u2_3, u2_6, u2_25, u2_30, u2_44, u2_49, u2_87, u2_93, u2_100]
u3 = [u3_0, u3_54, u3_56, u3_71, u3_80, u3_88, u3_100]
u4 = [u4_49, u4_54, u4_55, u4_60, u4_61, u4_67, u4_73, u4_75, u4_100]

us = [u1, u2, u3, u4]

In [11]:
prob += e, "Objective Function"

# Add constraints:

# preferations
prob += u1_48 + u2_87 + u3_0 + u4_75 == u1_76 + u2_6 + u3_100 + u4_60, "3 incomparable with 17"
prob += u1_74 + u2_25 + u3_80 + u4_49 >= u1_87 + u2_3 + u3_100 + u4_61 + e, "11 prefered over 14"
prob += u1_60 + u2_93 + u3_0 + u4_73 == u1_74 + u2_25 + u3_80 + u4_49 , "0 incomparable with 11"
prob += u1_69 + u2_49 + u3_56 + u4_61 >= u1_71 + u2_25 + u3_88 + u4_67 + e, "13 prefered over 25"
prob += u1_64 + u2_44 + u3_54 + u4_54 >= u1_65 + u2_30 + u3_71 + u4_55 + e, "7 prefered over 8"

# normalizations
prob += u1_32 + u2_3 + u3_0 + u4_49 == 1, "normalization"
prob += u1_100 == 0, "u1_min"
prob += u2_100 == 0, "u2_min"
prob += u3_100 == 0, "u3_min"
prob += u4_100 == 0, "u4_min"

# non-negativity - already set in LpVariable
# for u in us:
#     for a in u:
#         prob += a >= 0

# monotonicity
for u in us:
    for a, b in zip(u, u[1:]):
        prob += a >= b


In [12]:
prob

MaximizeEpsilon:
MAXIMIZE
1*e + 0
SUBJECT TO
3_incomparable_with_17: u1_48 - u1_76 - u2_6 + u2_87 + u3_0 - u3_100 - u4_60
 + u4_75 = 0

11_prefered_over_14: - e + u1_74 - u1_87 - u2_03 + u2_25 - u3_100 + u3_80
 + u4_49 - u4_61 >= 0

0_incomparable_with_11: u1_60 - u1_74 - u2_25 + u2_93 + u3_0 - u3_80 - u4_49
 + u4_73 = 0

13_prefered_over_25: - e + u1_69 - u1_71 - u2_25 + u2_49 + u3_56 - u3_88
 + u4_61 - u4_67 >= 0

7_prefered_over_8: - e + u1_64 - u1_65 - u2_30 + u2_44 + u3_54 - u3_71 + u4_54
 - u4_55 >= 0

u1_min: u1_100 = 0

u2_min: u2_100 = 0

u3_min: u3_100 = 0

u4_min: u4_100 = 0

_C1: u1_32 - u1_48 >= 0

_C2: u1_48 - u1_60 >= 0

_C3: u1_60 - u1_64 >= 0

_C4: u1_64 - u1_65 >= 0

_C5: u1_65 - u1_69 >= 0

_C6: u1_69 - u1_71 >= 0

_C7: u1_71 - u1_74 >= 0

_C8: u1_74 - u1_76 >= 0

_C9: u1_76 - u1_87 >= 0

_C10: - u1_100 + u1_87 >= 0

_C11: u2_03 - u2_6 >= 0

_C12: - u2_25 + u2_6 >= 0

_C13: u2_25 - u2_30 >= 0

_C14: u2_30 - u2_44 >= 0

_C15: u2_44 - u2_49 >= 0

_C16: u2_49 - u2_87 >=

In [13]:
# Solve the problem
prob.solve()
print("Status:", LpStatus[prob.status])

# Print the optimal production amount of x and y
for v in prob.variables():
    print(v.name, "=", v.varValue)

Status: Unbounded
e = 1.6e-11
u1_100 = 0.0
u1_32 = 0.0
u1_48 = 0.0
u1_60 = 1e-12
u1_64 = 2e-12
u1_65 = -2e-12
u1_69 = -1e-12
u1_71 = 0.0
u1_74 = 1e-12
u1_76 = 2e-12
u1_87 = 0.0
u2_03 = 0.0
u2_100 = 0.0
u2_25 = 0.0
u2_30 = 0.0
u2_44 = 1e-12
u2_49 = 1e-12
u2_6 = 1e-12
u2_87 = 0.0
u2_93 = 0.0
u3_0 = 4e-12
u3_100 = 0.0
u3_54 = 5e-12
u3_56 = 5e-12
u3_71 = 0.0
u3_80 = 0.0
u3_88 = 0.0
u4_100 = 0.0
u4_49 = 4e-12
u4_54 = 5e-12
u4_55 = 0.0
u4_60 = 1e-12
u4_61 = 4e-12
u4_67 = 0.0
u4_73 = 0.0
u4_75 = 0.0


In [14]:
f1_y = [u.varValue for u in u1]
f2_y = [u.varValue for u in u2]
f3_y = [u.varValue for u in u3]
f4_y = [u.varValue for u in u4]

print(f1_y)
print(f2_y)
print(f3_y)
print(f4_y)

[0.0, 0.0, 1e-12, 2e-12, -2e-12, -1e-12, 0.0, 1e-12, 2e-12, 0.0, 0.0]
[0.0, 1e-12, 0.0, 0.0, 1e-12, 1e-12, 0.0, 0.0, 0.0]
[4e-12, 5e-12, 5e-12, 0.0, 0.0, 0.0, 0.0]
[4e-12, 5e-12, 0.0, 1e-12, 4e-12, 0.0, 0.0, 0.0, 0.0]
